# NLP Homework 4 Programming Assignment

In this assignment, we will train and evaluate a neural model to tag the parts of speech in a sentence.
We will also implement several improvements to the model to test its performance.

We will be using English text from the Wall Street Journal, marked with POS tags such as `NNP` (proper noun) and `DT` (determiner).

## Building a POS Tagger

### Setup

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

random.seed(1)

### Preparing Data
We collect the data in the following cell from the `train.txt` and `test.txt` files.  
For `train.txt`, we read the word and tag sequences for each sentence. We then create an 80-20 train-val split on this data for training and evaluation purpose.

Finally, we are interested in our accuracy on `test.txt`, so we prepare test data from this file.

In [2]:
def load_tag_data(tag_file):
    all_sentences = []
    all_tags = []
    sent = []
    tags = []
    with open(tag_file, 'r') as f:
        for line in f:
            if line.strip() == "":
                all_sentences.append(sent)
                all_tags.append(tags)
                sent = []
                tags = []
            else:
                word, tag, _ = line.strip().split()
                sent.append(word)
                tags.append(tag)
    return all_sentences, all_tags

def load_txt_data(txt_file):
    all_sentences = []
    sent = []
    with open(txt_file, 'r') as f:
        for line in f:
            if(line.strip() == ""):
                all_sentences.append(sent)
                sent = []
            else:
                word = line.strip()
                sent.append(word)
    return all_sentences

train_sentences, train_tags = load_tag_data('train.txt')
test_sentences = load_txt_data('test.txt')

unique_tags = set([tag for tag_seq in train_tags for tag in tag_seq])

# Create train-val split from train data
train_val_data = list(zip(train_sentences, train_tags))
random.shuffle(train_val_data)
split = int(0.8 * len(train_val_data))
training_data = train_val_data[:split]
val_data = train_val_data[split:]

print("Train Data: ", len(training_data))
print("Val Data: ", len(val_data))
print("Test Data: ", len(test_sentences))
print("Total tags: ", len(unique_tags))

Train Data:  7148
Val Data:  1788
Test Data:  2012
Total tags:  44


### Word-to-Index and Tag-to-Index mapping
In order to work with text in Tensor format, we need to map each word to an index.

In [3]:
word_to_idx = {}
for sent in train_sentences:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

for sent in test_sentences:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
            
tag_to_idx = {}
for tag in unique_tags:
    if tag not in tag_to_idx:
        tag_to_idx[tag] = len(tag_to_idx)

idx_to_tag = {}
for tag in tag_to_idx:
    idx_to_tag[tag_to_idx[tag]] = tag
print(train_sentences)
print(word_to_idx)
print("Total tags", len(tag_to_idx))
print("Vocab size", len(word_to_idx))

[['Confidence', 'in', 'the', 'pound', 'is', 'widely', 'expected', 'to', 'take', 'another', 'sharp', 'dive', 'if', 'trade', 'figures', 'for', 'September', ',', 'due', 'for', 'release', 'tomorrow', ',', 'fail', 'to', 'show', 'a', 'substantial', 'improvement', 'from', 'July', 'and', 'August', "'s", 'near-record', 'deficits', '.'], ['Chancellor', 'of', 'the', 'Exchequer', 'Nigel', 'Lawson', "'s", 'restated', 'commitment', 'to', 'a', 'firm', 'monetary', 'policy', 'has', 'helped', 'to', 'prevent', 'a', 'freefall', 'in', 'sterling', 'over', 'the', 'past', 'week', '.'], ['But', 'analysts', 'reckon', 'underlying', 'support', 'for', 'sterling', 'has', 'been', 'eroded', 'by', 'the', 'chancellor', "'s", 'failure', 'to', 'announce', 'any', 'new', 'policy', 'measures', 'in', 'his', 'Mansion', 'House', 'speech', 'last', 'Thursday', '.'], ['This', 'has', 'increased', 'the', 'risk', 'of', 'the', 'government', 'being', 'forced', 'to', 'increase', 'base', 'rates', 'to', '16', '%', 'from', 'their', 'curre

In [0]:
def prepare_sequence(sent, idx_mapping):
    idxs = [idx_mapping[word] for word in sent]
    return torch.tensor(idxs, dtype=torch.long)

### Set up model
We will build and train a Basic POS Tagger which is an LSTM model to tag the parts of speech in a given sentence.


First we need to define some default hyperparameters.

In [0]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 200
LEARNING_RATE = 0.1
LSTM_LAYERS = 2
DROPOUT = 1
EPOCHS = 2

### Define Model

The model takes as input a sentence as a tensor in the index space. This sentence is then converted to embedding space where each word maps to its word embedding. The word embeddings is learned as part of the model training process. 

These word embeddings act as input to the LSTM which produces a hidden state. This hidden state is then passed to a Linear layer that produces the probability distribution for the tags of every word. The model will output the tag with the highest probability for a given word.

In [0]:
class BasicPOSTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(BasicPOSTagger, self).__init__()
        #############################################################################
        # TODO: Define and initialize anything needed for the forward pass.
        # You are required to create a model with:
        # an embedding layer: that maps words to the embedding space
        # an LSTM layer: that takes word embeddings as input and outputs hidden states
        # a Linear layer: maps from hidden state space to tag space
        #############################################################################
        self.hidden_dim = hidden_dim
        self.word_embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim)
        self.hiddenTotag = nn.Linear(hidden_dim, tagset_size)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def forward(self, sentence):
        tag_scores = None
        #############################################################################
        # TODO: Implement the forward pass.
        # Given a tokenized index-mapped sentence as the argument, 
        # compute the corresponding scores for tags
        # returns:: tag_scores (Tensor)
        #############################################################################
        ebdngs = self.word_embedding(sentence)
        lstm_output, _ = self.lstm(ebdngs.view(len(sentence),1,-1))
        tag_seq = self.hiddenTotag(lstm_output.view(len(sentence),-1))
        tag_scores  = F.log_softmax(tag_seq, dim=1)
        
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        return tag_scores

### Training

We define train and evaluate procedures that allow us to train our model using our created train-val split.

In [0]:
def train(epoch, model, loss_function, optimizer):
    train_loss = 0
    train_examples = 0
    for sentence, tags in training_data:
        #############################################################################
        # TODO: Implement the training loop
        # Hint: you can use the prepare_sequence method for creating index mappings 
        # for sentences. Find the gradient with respect to the loss and update the
        # model parameters using the optimizer.
        #############################################################################
        model.zero_grad()
        sen_input = prepare_sequence(sentence,word_to_idx)
        targets = prepare_sequence(tags,tag_to_idx)
        #print(targets.shape)
        tag_scores = model(sen_input)
        #print(tag_scores.shape)
        loss = loss_function(tag_scores,targets)
        train_examples+=1
        loss.backward()
        optimizer.step() 
        train_loss += loss
        train_examples += len(targets)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
    
    avg_train_loss = train_loss / train_examples
    avg_val_loss, val_accuracy = evaluate(model, loss_function, optimizer)
        
    print("Epoch: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\t Val Accuracy: {:.0f}".format(epoch, 
                                                                      EPOCHS, 
                                                                      avg_train_loss, 
                                                                      avg_val_loss,
                                                                      val_accuracy))

def evaluate(model, loss_function, optimizer):
  # returns:: avg_val_loss (float)
  # returns:: val_accuracy (float)
    val_loss = 0
    correct = 0
    val_examples = 0
    with torch.no_grad():
        for sentence, tags in val_data:
            #############################################################################
            # TODO: Implement the evaluate loop
            # Find the average validation loss along with the validation accuracy.
            # Hint: To find the accuracy, argmax of tag predictions can be used.
            #############################################################################
            # model.zero_grad()
            sen_input = prepare_sequence(sentence, word_to_idx)
            targets = prepare_sequence(tags,tag_to_idx)
            tag_scores = model(sen_input)
            _, indices = torch.max(tag_scores,1)
            val_loss += loss_function(tag_scores,targets)
            correct += torch.sum(indices == torch.LongTensor(targets))
            val_examples += len(targets)
            
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
    val_accuracy = 100. * correct / val_examples
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, val_accuracy


In [8]:
#############################################################################
# TODO: Initialize the model, optimizer and the loss function
#############################################################################
model = BasicPOSTagger(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_idx.keys()),len(tag_to_idx.keys()))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    train(epoch, model, loss_function, optimizer)

Epoch: 1/2	Avg Train Loss: 0.0381	Avg Val Loss: 0.0232	 Val Accuracy: 85
Epoch: 2/2	Avg Train Loss: 0.0155	Avg Val Loss: 0.0166	 Val Accuracy: 90


You should get a performance of **at least 80%** on the validation set for the BasicPOSTagger.

Let us now write a method to save our predictions for the test set.

In [0]:
def test():
    val_loss = 0
    correct = 0
    val_examples = 0
    predicted_tags = []
    with torch.no_grad():
        for sentence in test_sentences:
            #############################################################################
            # TODO: Implement the test loop
            # This method saves the predicted tags for the sentences in the test set.
            # The tags are first added to a list which is then written to a file for
            # submission. An empty string is added after every sequence of tags
            # corresponding to a sentence to add a newline following file formatting
            # convention, as has been done already.
            #############################################################################
            sen_in = prepare_sequence(sentence,word_to_idx)
            tag_scores = model(sen_in)
            _, indexes = torch.max(tag_scores,1)
            for i in range(len(indexes)):
              for key, value in tag_to_idx.items():
                if indexes[i] == value:
                    predicted_tags.append(key)
            
                    
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            predicted_tags.append("")
    print(predicted_tags)
    with open('test_labels.txt', 'w+') as f:
        for item in predicted_tags:
            f.write("%s\n" % item)
    

In [10]:
test()

['NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'NN', 'PRP$', 'NN', 'IN', 'VBN', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'VBG', 'POS', 'VBG', 'NN', '.', '', 'NNP', 'VBD', 'DT', 'NN', 'VBZ', 'IN', 'PRP', 'TO', 'VB', 'CD', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NNS', '.', '', 'DT', 'VBP', ',', 'IN', 'JJ', 'NNS', ',', 'DT', 'NN', 'POS', 'CD', 'JJ', 'NN', ',', 'DT', 'NN', 'NN', ',', 'NNP', 'NN', ',', 'VBN', 'VBG', 'JJ', 'IN', 'DT', 'NNS', 'CC', 'DT', 'NN', 'JJ', 'NN', '.', '', 'IN', 'DT', 'NNP', 'NN', ',', 'NNP', 'VBD', ',', 'PRP', 'VBZ', 'RB', 'NN', 'RB', 'IN', 'DT', 'NN', 'TO', 'VB', '.', '', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'VBZ', 'DT', 'NN', ',', 'VBD', ',', 'JJ', 'CC', 'NNS', 'NN', '.', '', 'NNP', 'NNS', 'NNP', 'VBD', 'VBN', 'TO', 'DT', 'NNS', 'NN', 'POS', 'NN', ',', 'VBD', 'DT', 'NN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NN', 'JJ', 'NNP', '.', '', 'NNP', 'NNP', ',', 'NNP', 'NNS', 'JJ', ',', 'NN', 'IN', 'NN', 'NN', 'IN', 'DT', 'NNP', 'NN


### Test accuracy
Evaluate your performance on the test data by submitting test_labels.txt generated by the method above and **report your test accuracy here**.

The test accuracy I got for BasicLSTM is 90.4

Imitate the above method to generate prediction for validation data.
Create lists of words, tags predicted by the model and ground truth tags. 

Use these lists to carry out error analysis to find the top-10 types of errors made by the model.

In [0]:
#############################################################################
# TODO: Generate predictions from val data
# Create lists of words, tags predicted by the model and ground truth tags.
#############################################################################
def generate_predictions(model, test_sentences):
    # returns:: word_list (str list)
    # returns:: model_tags (str list)
    # returns:: gt_tags (str list)
    # Your code here
    word_list = []
    model_tags = []
    gt_tags = []
    for sentence,tag in test_sentences:
      sen_in = prepare_sequence(sentence,word_to_idx)
      # gt_tags.append(tag)
      # word_list.append(sentence)
      tag_scores = model(sen_in)
      _, indexes = torch.max(tag_scores,1)
      for i in range(len(indexes)):
        for key, value in tag_to_idx.items():
          if indexes[i] == value:
            model_tags.append(key)
      word_list.extend(sentence)
      gt_tags.extend(tag)

          # model_tags.append()
          #gt_tags.append(key) 
        #word_list.append(sentence[i])     
    return word_list, model_tags, gt_tags

#############################################################################
# TODO: Carry out error analysis
# From those lists collected from the above method, find the 
# top-10 tuples of (model_tag, ground_truth_tag, frequency, example words)
# sorted by frequency
#############################################################################
def error_analysis(word_list, model_tags, gt_tags):
    # returns: errors (list of tuples)
    # Your code here
    import collections
    d = collections.defaultdict(list)
    

    temp_wl=[]
    temp_ml=[]
    temp_gt=[]
    errors = []
    for word,pred,ground_truth in zip(word_list,model_tags,gt_tags):
      # to find the error we need to find where the prediction is not equal to ground truth
      if pred != ground_truth:
        temp_wl.append(word)
        temp_ml.append(pred)
        temp_gt.append(ground_truth)
        d[(pred,ground_truth)].append(word)

    
    combined_pred_gt = zip(temp_ml,temp_gt)
    cc = collections.Counter(combined_pred_gt)

    for (pred,ground_truth), count in cc.most_common(10):
      freq = float(count/sum(cc.values()))
      errors.append((pred,ground_truth,freq,d[(pred,ground_truth)][:10]))



    
    
    
    return errors

In [17]:
word_list,model_tags,gt_tags = generate_predictions(model,val_data)
print(len(word_list),len(model_tags),len(gt_tags))
errors = error_analysis(word_list,model_tags,gt_tags)
for error in errors:
  print(error)

41851 41851 41851
('JJ', 'NN', 0.05558086560364465, ['democratization', 'sign', 'file', 'weekly', 'black', 'current', 'crisis', 'discrimination', 'common', 'official'])
('NN', 'JJ', 0.05558086560364465, ['literary', 'insolvent', 'electric', 'Miami-based', 'ultimate', 'penny-brokerage', 'labor-management', 'flu-like', 'odd', 'five-session'])
('NN', 'NNP', 0.05079726651480638, ['Sununu', 'SNET', 'Cordis', 'Lyneses', 'Information', 'Schering-Plough', 'business', 'Houston', 'Roe', 'Round'])
('JJ', 'NNP', 0.04031890660592255, ['Extension', 'Education', 'EC', 'Rent', 'J.P.', 'Joaquin', 'Agency', 'Darman', 'Dayton', 'Fireman'])
('NN', 'NNS', 0.03895216400911162, ['twists', 'foundations', 'piers', 'surprises', 'aspirations', 'Offices', 'associations', 'troops', 'shots', 'symptoms'])
('NNP', 'NN', 0.03530751708428246, ['collapse', 'Circulation', 'corporation', 'aim', 'sight', 'inquiry', 'Unemployment', 'TOPIC', 'fluoride', 'gourmet'])
('NNP', 'JJ', 0.025968109339407745, ['historic', 'fetal-tiss

### Error analysis
**Report your findings here.**  
What kinds of errors did the model make and why do you think it made them?

The model makes error in predicting the NN tag properly. If we look at the first row of the error, we can see that the model fail to predict the proper tag NN for the words, instead it predicted JJ tag for those words. I can see that the model predicted some nouns correctly but to some it has identified as adjective which i think is correct like "black". I think it might be because of the context in which they appear and their position in the sentence is what is confusing the model, as our model is unidirectional.

## Define a Character Level POS Tagger

We can use the character-level information present to augment our word embeddings. Words that end with -ing or -ly give quite a bit of information about their POS tags. To incorporate this information, we can run a character level LSTM on every word (treated as a tensor of characters, each mapped to character-index space) to create a character-level representation of the word. This representation can be concatenated with the word embedding (as in the BasicPOSTagger) to create a new word embedding that captures more information.

In [0]:
# Create char to index mapping
char_to_idx = {}
unique_chars = set()
MAX_WORD_LEN = 0

for sent in train_sentences:
    for word in sent:
        for c in word:
            unique_chars.add(c)
        if len(word) > MAX_WORD_LEN:
            MAX_WORD_LEN = len(word)

for c in unique_chars:
    char_to_idx[c] = len(char_to_idx)
char_to_idx[' '] = len(char_to_idx)

# New Hyperparameters
EMBEDDING_DIM = 12
HIDDEN_DIM = 12
LEARNING_RATE = 0.01
LSTM_LAYERS = 4
DROPOUT = 2
EPOCHS = 10
CHAR_EMBEDDING_DIM = 3
CHAR_HIDDEN_DIM = 3

In [0]:
class CharPOSTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, char_embedding_dim, 
                 char_hidden_dim, char_size, vocab_size, tagset_size):
        super(CharPOSTagger, self).__init__()
        #############################################################################
        # TODO: Define and initialize anything needed for the forward pass.
        # You are required to create a model with:
        # an embedding layer: that maps words to the embedding space
        # an char level LSTM: that finds the character level embedding for a word
        # an LSTM layer: that takes the combined embeddings as input and outputs hidden states
        # a Linear layer: maps from hidden state space to tag space
        #############################################################################
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.char_embedding = nn.Embedding(char_size, char_embedding_dim )
        self.charLSTM = nn.LSTM(char_embedding_dim, char_hidden_dim)
        self.lstm = nn.LSTM(embedding_dim+char_hidden_dim, hidden_dim)
        self.hiddenToTag = nn.Linear(hidden_dim, tagset_size)

        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def forward(self, sentence, chars):
        tag_scores = None
        #############################################################################
        # TODO: Implement the forward pass.
        # Given a tokenized index-mapped sentence and a character sequence as the arguments, 
        # find the corresponding scores for tags
        # returns:: tag_scores (Tensor)
        #############################################################################
        embeddings = self.word_embedding(sentence)
        char_hidden_result = []
        for char in chars:
          char_embedding = self.char_embedding(char)
          _, (char_hidden_state, char_cell_state) = self.charLSTM(char_embedding.view(len(char),1,-1))
          char_word_hidden = char_hidden_state.view(-1)
          char_hidden_result.append(char_word_hidden)
        char_hidden_result = torch.stack(tuple(char_hidden_result))
        combined_embedding = torch.cat((embeddings,char_hidden_result),1)
        lstm_result, _ = self.lstm(combined_embedding.view(len(sentence),1,-1))
        tag_s = self.hiddenToTag(lstm_result.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_s,dim=1)

        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        return tag_scores

def train_char(epoch, model, loss_function, optimizer):
    train_loss = 0
    train_examples = 0
    for sentence, tags in training_data:
        #############################################################################
        # TODO: Implement the training loop
        # Hint: you can use the prepare_sequence method for creating index mappings 
        # for sentences as well as character sequences. Find the gradient with 
        # respect to the loss and update the model parameters using the optimizer.
        #############################################################################
        words = []
        for word in sentence:
          words.append(prepare_sequence(word,char_to_idx))
        # words = prepare_sequence(sentence,char_to_idx)
        #print(words)
        sentence_in = prepare_sequence(sentence,word_to_idx)
        targets = prepare_sequence(tags,tag_to_idx)
        model.zero_grad()
        tag_scores = model(sentence_in,words)
        loss = loss_function(tag_scores,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss
        train_examples += len(targets)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
    
    avg_train_loss = train_loss / train_examples
    avg_val_loss, val_accuracy = evaluate_char(model, loss_function, optimizer)
        
    print("Epoch: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\t Val Accuracy: {:.0f}".format(epoch, 
                                                                      EPOCHS, 
                                                                      avg_train_loss, 
                                                                      avg_val_loss,
                                                                      val_accuracy))

def evaluate_char(model, loss_function, optimizer):
    # returns:: avg_val_loss (float)
    # returns:: val_accuracy (float)
    val_loss = 0
    correct = 0
    val_examples = 0
    with torch.no_grad():
        for sentence, tags in val_data:
            #############################################################################
            # TODO: Implement the evaluate loop
            # Find the average validation loss along with the validation accuracy.
            # Hint: To find the accuracy, argmax of tag predictions can be used.
            #############################################################################
            
            words = []
            for word in sentence:
              words.append(prepare_sequence(word,char_to_idx))
            sen_input = prepare_sequence(sentence, word_to_idx)
            targets = prepare_sequence(tags,tag_to_idx)
            tag_scores = model(sen_input,words)
            _, indices = torch.max(tag_scores,1)
            val_loss += loss_function(tag_scores,targets)
            correct += torch.sum(indices == torch.LongTensor(targets))
            val_examples += len(targets)
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
    val_accuracy = 100. * correct / val_examples
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, val_accuracy

In [7]:
#############################################################################
# TODO: Initialize the model, optimizer and the loss function
#############################################################################
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
#   model.cuda()

model = CharPOSTagger(EMBEDDING_DIM,HIDDEN_DIM,CHAR_EMBEDDING_DIM,CHAR_HIDDEN_DIM,
                       len(char_to_idx.keys()), len(word_to_idx.keys()), len(tag_to_idx.keys()))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)


import time
#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    start = time.time()
    train_char(epoch, model, loss_function, optimizer)
    print(f"time used in this epoch{epoch}: ", time.time() - start)

Epoch: 1/10	Avg Train Loss: 0.0279	Avg Val Loss: 0.0145	 Val Accuracy: 91
time used in this epoch1:  213.51745438575745
Epoch: 2/10	Avg Train Loss: 0.0098	Avg Val Loss: 0.0117	 Val Accuracy: 93
time used in this epoch2:  228.90641283988953
Epoch: 3/10	Avg Train Loss: 0.0064	Avg Val Loss: 0.0114	 Val Accuracy: 93
time used in this epoch3:  225.43528962135315
Epoch: 4/10	Avg Train Loss: 0.0049	Avg Val Loss: 0.0118	 Val Accuracy: 94
time used in this epoch4:  235.64571452140808
Epoch: 5/10	Avg Train Loss: 0.0041	Avg Val Loss: 0.0119	 Val Accuracy: 94
time used in this epoch5:  249.47587370872498
Epoch: 6/10	Avg Train Loss: 0.0036	Avg Val Loss: 0.0119	 Val Accuracy: 94
time used in this epoch6:  245.06821250915527
Epoch: 7/10	Avg Train Loss: 0.0033	Avg Val Loss: 0.0124	 Val Accuracy: 94
time used in this epoch7:  240.75135707855225
Epoch: 8/10	Avg Train Loss: 0.0030	Avg Val Loss: 0.0124	 Val Accuracy: 94
time used in this epoch8:  241.44244074821472
Epoch: 9/10	Avg Train Loss: 0.0029	Avg V

In [0]:
def test():
    val_loss = 0
    correct = 0
    val_examples = 0
    predicted_tags = []
    with torch.no_grad():
        for sentence in test_sentences:
            #############################################################################
            # TODO: Implement the test loop
            # This method saves the predicted tags for the sentences in the test set.
            # The tags are first added to a list which is then written to a file for
            # submission. An empty string is added after every sequence of tags
            # corresponding to a sentence to add a newline following file formatting
            # convention, as has been done already.
            #############################################################################
            words = []
            for word in sentence:
              words.append(prepare_sequence(word,char_to_idx))
            sen_input = prepare_sequence(sentence, word_to_idx)
            # targets = prepare_sequence(tags,tag_to_idx)
            tag_scores = model(sen_input,words)
            _, indexes = torch.max(tag_scores,1)
            for i in range(len(indexes)):
              for key, value in tag_to_idx.items():
                if indexes[i] == value:
                    predicted_tags.append(key)
                    
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            predicted_tags.append("")
    print(predicted_tags)
    with open('test_labels_char.txt', 'w+') as f:
        for item in predicted_tags:
            f.write("%s\n" % item)
    

In [9]:
test()

['NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'NN', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', 'VBZ', '.', '', 'NNP', 'VBD', 'DT', 'NN', 'VBZ', 'IN', 'PRP', 'TO', 'VB', 'CD', 'JJ', 'JJ', 'NNS', 'IN', 'DT', 'NNS', '.', '', 'DT', 'VBP', ',', 'IN', 'JJ', 'NNS', ',', 'DT', 'NN', 'POS', 'CD', 'JJ', 'NNS', ',', 'DT', 'NN', 'NN', ',', 'NNP', 'NN', ',', 'VBN', 'VBG', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'DT', 'NN', 'NNP', 'NN', '.', '', 'IN', 'DT', 'VBG', 'NN', ',', 'NNP', 'VBD', ',', 'PRP', 'VBZ', 'RB', 'VBN', 'CD', 'IN', 'DT', 'NNS', 'TO', 'NNP', '.', '', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'VBZ', 'DT', 'NN', ',', 'NNS', ',', 'JJ', 'CC', 'NNS', 'VBP', '.', '', 'NNP', 'NN', 'NNP', 'VBD', 'VBN', 'TO', 'DT', 'NNS', 'NN', 'POS', 'NN', ',', 'VBG', 'DT', 'NN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'VBP', 'JJ', 'NNP', '.', '', 'NNP', 'NN', ',', 'CD', 'NNS', 'JJ', ',', 'VBD', 'IN', 'NN', 'NN', 'IN', 'DT', 'NN

Tune your hyperparameters, to get a performance of **at least 85%** on the validation set for the CharPOSTagger.

### Test accuracy
Also evaluate your performance on the test data by submitting test_labels.txt and **report your test accuracy here**.

92.41

### Error analysis

In [0]:
#############################################################################
# TODO: Generate predictions from val data
# Create lists of words, tags predicted by the model and ground truth tags.
#############################################################################
def generate_predictions(model, test_sentences):
    # returns:: word_list (str list)
    # returns:: model_tags (str list)
    # returns:: gt_tags (str list)
    # Your code here
    word_list = []
    model_tags = []
    gt_tags = []
    for sentence,tag in test_sentences:
      words = []
      for word in sentence:
        words.append(prepare_sequence(word,char_to_idx))
      sen_in = prepare_sequence(sentence,word_to_idx)
      # gt_tags.append(tag)
      # word_list.append(sentence)
      tag_scores = model(sen_in,words)
      _, indexes = torch.max(tag_scores,1)
      for i in range(len(indexes)):
        for key, value in tag_to_idx.items():
          if indexes[i] == value:
            model_tags.append(key)
      word_list.extend(sentence)
      gt_tags.extend(tag)

          # model_tags.append()
          #gt_tags.append(key) 
        #word_list.append(sentence[i])     
    return word_list, model_tags, gt_tags

#############################################################################
# TODO: Carry out error analysis
# From those lists collected from the above method, find the 
# top-10 tuples of (model_tag, ground_truth_tag, frequency, example words)
# sorted by frequency
#############################################################################
def error_analysis(word_list, model_tags, gt_tags):
    # returns: errors (list of tuples)
    # Your code here
    import collections
    d = collections.defaultdict(list)
    

    temp_wl=[]
    temp_ml=[]
    temp_gt=[]
    errors = []
    for word,pred,ground_truth in zip(word_list,model_tags,gt_tags):
      # to find the error we need to find where the prediction is not equal to ground truth
      if pred != ground_truth:
        temp_wl.append(word)
        temp_ml.append(pred)
        temp_gt.append(ground_truth)
        d[(pred,ground_truth)].append(word)

    
    combined_pred_gt = zip(temp_ml,temp_gt)
    cc = collections.Counter(combined_pred_gt)

    for (pred,ground_truth), count in cc.most_common(10):
      freq = float(count/sum(cc.values()))
      errors.append((pred,ground_truth,freq,d[(pred,ground_truth)][:10]))
    return errors

In [11]:
word_list,model_tags,gt_tags = generate_predictions(model,val_data)
print(len(word_list),len(model_tags),len(gt_tags))
errors = error_analysis(word_list,model_tags,gt_tags)
for error in errors:
  print(error)

41851 41851 41851
('NN', 'NNP', 0.06224066390041494, ['Herman', 'Cordis', 'business', 'Willman', 'Strategic', 'STREET', 'Tela', 'Protestantism', 'Wilhite', 'Budweiser'])
('NN', 'JJ', 0.05726141078838174, ['fetal-tissue', 'ultimate', 'equitable', 'flu-like', 'five-session', 'much', 'unusable', 'net', 'image-building', 'arcane'])
('JJ', 'NN', 0.05228215767634855, ['panhandler', 'chief', 'drill', 'grade', 'weekly', 'black', 'current', 'common', 'cigar', 'gamma'])
('VBN', 'VBD', 0.04066390041493776, ['received', 'stepped', 'displayed', 'doubled', 'surveyed', 'closed', 'closed', 'insured', 'disclosed', 'called'])
('JJ', 'NNP', 0.036929460580912864, ['Joan', 'Sanford', 'Ground', 'PATOIS', 'Orwell', 'German', 'Long', 'Bruwer', 'American', 'Stella'])
('VBD', 'VBN', 0.036514522821576766, ['confiscated', 'contrived', 'established', 'covered', 'participated', 'featured', 'triggered', 'unleashed', 'executed', 'named'])
('WDT', 'IN', 0.036514522821576766, ['that', 'that', 'that', 'that', 'that', 't


**Report your findings here.**  
What kinds of errors does the character-level model make as compared to the original model, and why do you think it made them? 
- Interstingly the classifier here has made a mistake in recognizing proper noun, instead it has made a mistake in identifying adjectives as  Noun. so for example "3-share" is identified as noun instead of adjectice,while "Herman" is identified as Noun instead of proper noun. I think it is the same mistakes as done by basic lstm, but here the difference is that now it not able to properly tag determiners and puntuations. firstly i think the model is not converged very properly though getting a 93 per accuracy after 2 epoch. I should have trained it more. But here the reason might be char level information must have overweighted the sentence level information.

## Define a BiLSTM POS Tagger

A bidirectional LSTM that runs both left-to-right and right-to-left to represent dependencies between adjacent words in both directions and thus captures dependencies in both directions. 

In this part, you make your model bidirectional. 

In addition, you should implement one of these modifications to improve the model's performance:
- Tune the model hyperparameters. Try at least 5 different combinations of parameters. For example:
    - number of LSTM layers
    - number of hidden dimensions
    - number of word embedding dimensions
    - dropout rate
    - learning rate
- Switch to pre-trained Word Embeddings instead of training them from scratch. Try at least one different embedding method. For example:
    - [Glove](https://nlp.stanford.edu/projects/glove/)
    - [Fast Text](https://fasttext.cc/docs/en/english-vectors.html)
- Implement a different model architecture. Try at least one different architecture. For example:
    - adding a conditional random field on top of the LSTM
    - adding Viterbi decoding to the model

In [28]:
!pip install torchtext==0.5.0

     |████████████████████████████████| 81kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 21.7MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [26]:
import torchtext as text 
print(text.__version__)
# ## 87 percent efficiency with these hyperparam
# EMBEDDING_DIM = 200
# DROPOUT = 0.1
# HIDDEN_DIM = 16
# LEARNING_RATE = 0.01
# ISBIDERECTIONAL = True
# LSTM_LAYERS = 4
# EPOCHS = 10
# ## 86 percent efficiency with these hyperparam
# EMBEDDING_DIM = 200
# DROPOUT = 0.1
# HIDDEN_DIM = 100
# LEARNING_RATE = 0.01
# ISBIDERECTIONAL = True
# LSTM_LAYERS = 4
# EPOCHS = 10
## 82 percent efficiency with these hyperparam
# EMBEDDING_DIM = 50
# DROPOUT = 0.1
# HIDDEN_DIM = 25
# LEARNING_RATE = 0.01
# ISBIDERECTIONAL = True
# LSTM_LAYERS = 4
# EPOCHS = 10
# 
#
#73 accuracy.. it decreased with these hyperparam
# EMBEDDING_DIM = 300
# DROPOUT = 0.1
# HIDDEN_DIM = 200
# LEARNING_RATE = 0.01
# ISBIDERECTIONAL = True
# LSTM_LAYERS = 4
# EPOCHS = 10
#
# 91
#
# EMBEDDING_DIM = 200
# DROPOUT = 0.2
# HIDDEN_DIM = 32
# LEARNING_RATE = 0.001
# ISBIDERECTIONAL = True
# LSTM_LAYERS = 2
# EPOCHS = 10
#
# 95
EMBEDDING_DIM = 200
DROPOUT = 0.2
HIDDEN_DIM = 32
LEARNING_RATE = 0.001
ISBIDERECTIONAL = True
LSTM_LAYERS = 2
EPOCHS = 10


0.5.0


## Hyperparameter Analysis

I observed that learning rate and drop out effects a lot in terms of training the model and the validation accuracy is impacted by. Higher learning rate means penalizing the weights more and does the model quickly converge but the accuracy dropped, so i started decreasing the learning rate and at dropout of 0.2 i was able to hit the accuracy on validation above 90 percente as well as in test data. Please see in below cells for the accuracy. I used Glove embedding with 200 dimensions, i tried different other dimensions, but all other leads to poor performance and therefor i choose 200 finally.

In [22]:
! ls .vector_cache/

glove.6B.100d.txt  glove.6B.200d.txt.pt  glove.6B.50d.txt
glove.6B.200d.txt  glove.6B.300d.txt	 glove.6B.zip


In [0]:
class BiLSTMPOSTagger(nn.Module):
    # NOTE: you may have to modify these function headers to include your 
    # modification, e.g. adding a parameter for embeddings data
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(BiLSTMPOSTagger, self).__init__()
        #############################################################################
        # TODO: Define and initialize anything needed for the forward pass.
        # You are required to create a model with:
        # an embedding layer: that maps words to the embedding space
        # a BiLSTM layer: that takes word embeddings as input and outputs hidden states
        # a Linear layer: maps from hidden state space to tag space
        #############################################################################
        self.gl = text.vocab.GloVe(name='6B', dim = embedding_dim)
        DROPOUT = 0.2
        # print(DROPOUT)
        self.dropout = nn.Dropout(DROPOUT)
        # if LSTM_LAYERS <1:
        #   DROPOUT = 0
        self.lstm = nn.LSTM(embedding_dim, hidden_size=hidden_dim,num_layers=LSTM_LAYERS,
                            dropout = DROPOUT if LSTM_LAYERS >1 else 0, bidirectional=ISBIDERECTIONAL)
        # if ISBIDERECTIONAL:
        #   hidden_dim = hidden_dim*2
        self.hiddenToTag = nn.Linear(hidden_dim*2 if ISBIDERECTIONAL else hidden_dim,tagset_size)
        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################

    def forward(self, sentence):
        tag_scores = None
        #############################################################################
        # TODO: Implement the forward pass.
        # Given a tokenized index-mapped sentence as the argument, 
        # find the corresponding scores for tags
        # returns:: tag_scores (Tensor)
        #############################################################################
        # print(dir(self.gl))
        
        embeddings = self.gl.get_vecs_by_tokens(sentence, lower_case_backup=True)
        lstm_result,_ = self.lstm(embeddings.view(len(sentence),1,-1))
        tag_s = self.hiddenToTag(lstm_result.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_s,dim=1)

        #############################################################################
        #                             END OF YOUR CODE                              #
        #############################################################################
        return tag_scores
    

In [0]:
def train(epoch, model, loss_function, optimizer):
  train_loss= 0
  train_examples = 0
  for sentence, tags in training_data:
    model.zero_grad()
    sentence_in = sentence
    targets = prepare_sequence(tags,tag_to_idx)
    tag_scores = model(sentence_in)
    loss = loss_function(tag_scores,targets)
    loss.backward()
    optimizer.step()
    train_loss+=loss.cpu().detach().numpy()
    train_examples+=len(targets.cpu().detach().numpy())
  avg_train_loss = train_loss / train_examples
  avg_val_loss, val_accuracy = evaluate(model, loss_function, optimizer)
  print("Epoch: {}/{}\tAvg Train Loss: {:.4f}\tAvg Val Loss: {:.4f}\t Val Accuracy: {:.0f}".format(epoch, 
                                                                      EPOCHS, 
                                                                      avg_train_loss, 
                                                                      avg_val_loss,
                                                                      val_accuracy))

def evaluate(model, loss_function, optimizer):
    # returns:: avg_val_loss (float)
    # returns:: val_accuracy (float)
    val_loss = 0
    correct = 0
    val_examples = 0
    with torch.no_grad():
        for sentence, tags in val_data:
          sentence_in = sentence
          targets = prepare_sequence(tags, tag_to_idx)
          tag_scores = model(sentence_in)
          _, indexes = torch.max(tag_scores,1)
          loss = loss_function(tag_scores,targets)
          val_loss += loss.cpu().detach().numpy()
          correct += (torch.sum(indexes == torch.LongTensor(targets)).cpu().detach().numpy())
          val_examples += len(targets.cpu().detach().numpy())
    val_accuracy = 100. * correct / val_examples
    avg_val_loss = val_loss / val_examples
    return avg_val_loss, val_accuracy



In [25]:
#############################################################################
# TODO: Initialize the model, optimizer and the loss function
#############################################################################
import time

model = BiLSTMPOSTagger(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, 
                        vocab_size=len(word_to_idx.keys()), tagset_size=len(tag_to_idx.keys()))
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)


#############################################################################
#                             END OF YOUR CODE                              #
#############################################################################
for epoch in range(1, EPOCHS + 1): 
    start = time.time()
    train(epoch, model, loss_function, optimizer)
    print(f"time used for epoch{epoch}: ",time.time() - start)

Epoch: 1/10	Avg Train Loss: 0.0306	Avg Val Loss: 0.0148	 Val Accuracy: 90
time used for epoch1:  175.20788526535034
Epoch: 2/10	Avg Train Loss: 0.0114	Avg Val Loss: 0.0104	 Val Accuracy: 93
time used for epoch2:  173.2424259185791
Epoch: 3/10	Avg Train Loss: 0.0082	Avg Val Loss: 0.0089	 Val Accuracy: 94
time used for epoch3:  175.07934665679932
Epoch: 4/10	Avg Train Loss: 0.0067	Avg Val Loss: 0.0084	 Val Accuracy: 94
time used for epoch4:  175.2431936264038
Epoch: 5/10	Avg Train Loss: 0.0058	Avg Val Loss: 0.0078	 Val Accuracy: 95
time used for epoch5:  174.24291110038757
Epoch: 6/10	Avg Train Loss: 0.0052	Avg Val Loss: 0.0076	 Val Accuracy: 95
time used for epoch6:  174.9787232875824
Epoch: 7/10	Avg Train Loss: 0.0046	Avg Val Loss: 0.0076	 Val Accuracy: 95
time used for epoch7:  177.15688467025757
Epoch: 8/10	Avg Train Loss: 0.0042	Avg Val Loss: 0.0074	 Val Accuracy: 95
time used for epoch8:  174.78469514846802
Epoch: 9/10	Avg Train Loss: 0.0040	Avg Val Loss: 0.0073	 Val Accuracy: 95
t

Your modified model should get a performance of **at least 90%** on the validation set.

### Test accuracy
Also evaluate your performance on the test data by submitting test_labels.txt and **report your test accuracy here**.

94.56



In [0]:
def test():
    val_loss = 0
    correct = 0
    val_examples = 0
    predicted_tags = []
    with torch.no_grad():
        for sentence in test_sentences:
            #############################################################################
            # TODO: Implement the test loop
            # This method saves the predicted tags for the sentences in the test set.
            # The tags are first added to a list which is then written to a file for
            # submission. An empty string is added after every sequence of tags
            # corresponding to a sentence to add a newline following file formatting
            # convention, as has been done already.
            #############################################################################
            # sen_in = prepare_sequence(sentence,word_to_idx)
            tag_scores = model(sentence)
            _, indexes = torch.max(tag_scores,1)
            for i in range(len(indexes)):
              for key, value in tag_to_idx.items():
                if indexes[i] == value:
                    predicted_tags.append(key)
            
                    
            #############################################################################
            #                             END OF YOUR CODE                              #
            #############################################################################
            predicted_tags.append("")
    print(predicted_tags)
    with open('test_labels_bilstm.txt', 'w+') as f:
        for item in predicted_tags:
            f.write("%s\n" % item)
    

In [29]:
test()

['NNP', 'NNP', 'NNP', 'POS', 'CD', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'VBG', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', 'NNS', '.', '', 'NNP', 'VBD', 'DT', 'NN', 'VBZ', 'IN', 'PRP', 'TO', 'NN', 'CD', 'JJ', 'JJ', 'NNS', 'IN', 'DT', 'NNS', '.', '', 'DT', 'VBP', ',', 'IN', 'JJ', 'NNS', ',', 'DT', 'NN', 'POS', 'CD', 'JJ', 'NNS', ',', 'DT', 'NN', 'NN', ',', 'CC', 'NN', ',', 'VBG', 'VBG', 'NNS', 'IN', 'DT', 'NN', 'CC', 'DT', 'NNP', 'NNP', 'NN', '.', '', 'IN', 'DT', 'VBG', 'NN', ',', 'NNP', 'VBD', ',', 'PRP', 'VBZ', 'RB', 'VBN', 'CD', 'IN', 'DT', 'NN', 'TO', 'NNP', '.', '', 'NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'VBZ', 'DT', 'NN', ',', 'NNS', ',', 'JJ', 'CC', 'NNS', 'VBP', '.', '', 'NNP', 'NNP', 'NNP', 'VBD', 'VBN', 'TO', 'DT', 'NN', 'NN', 'POS', 'NN', ',', 'VBG', 'DT', 'NN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'NN', 'NNP', '.', '', 'NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'IN', 'IN', 'NN', 'NN', 'IN', 'DT', 'NNP'

### Error analysis
**Report your findings here.**  
Compare the top-10 errors made by this modified model with the errors made by the model from part (a). 
If you tried multiple hyperparameter combinations, choose the model with the highest validation data accuracy.
What errors does the original model make as compared to the modified model, and why do you think it made them? 

Feel free to reuse the methods defined above for this purpose.

- Here as we can see the error analysis at the end of the notebook, the model is making mistakes in predicting NN, NNP, Verb. Looks like here the model is overweighting because now here the model is trying to identify the noun as proper noun, which could happen if it is tending towards overfitting.But the model is able to learn char level and also able to understand the position of a tag in the sentence in both direction and hence we can see the the tags are predicted better in BiLSTM

In [0]:
#############################################################################
# TODO: Generate predictions from val data
# Create lists of words, tags predicted by the model and ground truth tags.
#############################################################################
def generate_predictions(model, test_sentences):
    # returns:: word_list (str list)
    # returns:: model_tags (str list)
    # returns:: gt_tags (str list)
    # Your code here
    word_list = []
    model_tags = []
    gt_tags = []
    for sentence,tag in test_sentences:
      
      # sen_in = prepare_sequence(sentence,word_to_idx)
      # gt_tags.append(tag)
      # word_list.append(sentence)
      tag_scores = model(sentence)
      _, indexes = torch.max(tag_scores,1)
      for i in range(len(indexes)):
        for key, value in tag_to_idx.items():
          if indexes[i] == value:
            model_tags.append(key)
      word_list.extend(sentence)
      gt_tags.extend(tag)

          # model_tags.append()
          #gt_tags.append(key) 
        #word_list.append(sentence[i])     
    return word_list, model_tags, gt_tags

#############################################################################
# TODO: Carry out error analysis
# From those lists collected from the above method, find the 
# top-10 tuples of (model_tag, ground_truth_tag, frequency, example words)
# sorted by frequency
#############################################################################
def error_analysis(word_list, model_tags, gt_tags):
    # returns: errors (list of tuples)
    # Your code here
    import collections
    d = collections.defaultdict(list)
    

    temp_wl=[]
    temp_ml=[]
    temp_gt=[]
    errors = []
    for word,pred,ground_truth in zip(word_list,model_tags,gt_tags):
      # to find the error we need to find where the prediction is not equal to ground truth
      if pred != ground_truth:
        temp_wl.append(word)
        temp_ml.append(pred)
        temp_gt.append(ground_truth)
        d[(pred,ground_truth)].append(word)

    
    combined_pred_gt = zip(temp_ml,temp_gt)
    cc = collections.Counter(combined_pred_gt)

    for (pred,ground_truth), count in cc.most_common(10):
      freq = float(count/sum(cc.values()))
      errors.append((pred,ground_truth,freq,d[(pred,ground_truth)][:10]))



    
    
    
    return errors

In [31]:
word_list,model_tags,gt_tags = generate_predictions(model,val_data)
print(len(word_list),len(model_tags),len(gt_tags))
errors = error_analysis(word_list,model_tags,gt_tags)
for error in errors:
  print(error)

41851 41851 41851
('NNP', 'NN', 0.09544573643410853, ['province', 'press', 'cup', 'city', 'mail', 'catalog', 'common', 'predecessor', 'bloc', 'gamma'])
('NN', 'NNP', 0.09544573643410853, ['Extension', 'Service', 'Information', 'Agency', 'business', 'Index', 'Market', 'USDA', 'Ground', 'Round'])
('NN', 'JJ', 0.061046511627906974, ['nonprofit', 'adamant', 'ultimate', 'five-session', 'usual', 'formal', 'pure', 'anti-Sandinista', 'excessive', 'electronic'])
('JJ', 'NN', 0.05329457364341085, ['overhead', 'span', 'estuarian', 'rent', 'black', 'replacement', 'current', 'sight', 'eclectic', 'combination'])
('NNP', 'JJ', 0.041666666666666664, ['universal', 'historic', 'direct', 'nuclear', 'former', 'national', '30-share', 'second', 'arcane', 'due'])
('JJ', 'NNP', 0.03488372093023256, ['mare-COOR', 'Independent', 'German', 'Long', 'Economic', 'General', 'Legal', 'General', 'American', 'National'])
('VBN', 'VBD', 0.029554263565891473, ['stepped', 'displayed', 'decreased', 'closed', 'called', 'eme